# Sourcing data and handling it

In [1]:
import os.path
import time
            # data sourcing:
import csv
            # databases:
import sqlite3
            # data toolkit:
import numpy as np

import pandas as pd
            # plotting
import matplotlib.pyplot as plt
%matplotlib inline  
import seaborn as sns

from sklearn import datasets

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate, KFold
from sklearn.model_selection import learning_curve

from sklearn.preprocessing import LabelEncoder

from sklearn.multiclass import OneVsOneClassifier
from sklearn.utils import shuffle

In [20]:
# check raw data archive path extant:
atp_data_path = os.path.join('..','raw_data','archive')
assert os.path.exists(atp_data_path)

In [24]:
# check various data files expected are extant:
matches_data_path = os.path.join(atp_data_path, 'all_matches_1st50k.csv')
assert os.path.exists(matches_data_path)

matches_data_path_latest = os.path.join(atp_data_path, 'all_matches_1st50k_latest.csv')
assert os.path.exists(matches_data_path_latest)

matches_data_path_full = os.path.join(atp_data_path, 'all_matches.csv')
assert os.path.exists(matches_data_path_full)


In [2]:
# download to the raw_data folder from the following url
dataset_url = 'https://www.kaggle.com/datasets/ehallmar/a-large-tennis-dataset-for-atp-and-itf-betting?select=all_tournaments.csv'

# unzip archive

# confirm .csv files x 7 are there:
assert os.path.isfile('../raw_data/archive/all_matches.csv'), 'all_matches.csv not found. download from url above'
assert os.path.isfile('../raw_data/archive/all_players.csv'), 'all_matches.csv not found. download from url above'
assert os.path.isfile('../raw_data/archive/all_tournaments.csv'), 'all_matches.csv not found. download from url above'
assert os.path.isfile('../raw_data/archive/betting_moneyline.csv'), 'all_matches.csv not found. download from url above'
assert os.path.isfile('../raw_data/archive/betting_spreads.csv'), 'all_matches.csv not found. download from url above'
assert os.path.isfile('../raw_data/archive/betting_totals.csv'), 'all_matches.csv not found. download from url above'
assert os.path.isfile('../raw_data/archive/countries.csv'), 'all_matches.csv not found. download from url above'


In [4]:

# needs care here, too big for memory 
matches_df = pd.read_csv(matches_data_path, nrows=50_000)
matches_df.head()

/tmp/ipykernel_58164/209196284.py:2: DtypeWarning: Columns (9,11,46,49) have mixed types. Specify dtype option on import or set low_memory=False.
  matches_df = pd.read_csv(matches_data_path, nrows=50_000)


,Unnamed: 0,start_date,end_date,location,court_surface,prize_money,currency,year,player_id,player_name,...,total_points,duration,player_victory,retirement,seed,won_first_set,doubles,masters,round_num,nation
0,0,2012-06-11,2012-06-17,Slovakia,Clay,30000.0,€,2012,adrian-partl,A. Partl,...,88.0,01:02:00,f,f,NaN,f,f,100,1,Slovakia
1,1,2012-06-11,2012-06-17,Slovakia,Clay,30000.0,€,2012,adrian-partl,A. Partl,...,195.0,02:43:00,t,f,NaN,t,f,100,0,Slovakia
2,2,2012-06-11,2012-06-17,Slovakia,Clay,30000.0,€,2012,agustin-velotti,A. Velotti,...,126.0,01:17:00,f,f,NaN,f,f,100,3,Slovakia
3,3,2012-06-11,2012-06-17,Slovakia,Clay,30000.0,€,2012,agustin-velotti_marcel-felder,NaN,...,NaN,NaN,f,f,NaN,t,t,100,4,Slovakia
4,4,2012-06-11,2012-06-17,Slovakia,Clay,30000.0,€,2012,alessandro-giannessi,A. Giannessi,...,194.0,02:32:00,t,f,1,t,f,100,3,Slovakia


In [5]:
matches_data_latest = pd.read_csv(matches_data_path_latest)


/tmp/ipykernel_58164/1687976180.py:1: DtypeWarning: Columns (1,6,8,10,12,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,50,53,54) have mixed types. Specify dtype option on import or set low_memory=False.
  matches_data_latest = pd.read_csv(matches_data_path_latest)


In [6]:

matches_data_latest.head()
# matches_data_latest.count()

,Unnamed: 0.1,Unnamed: 0,start_date,end_date,location,court_surface,prize_money,currency,year,player_id,...,total_points,duration,player_victory,retirement,seed,won_first_set,doubles,masters,round_num,nation
0,0.0,0,2012-06-11,2012-06-17,Slovakia,Clay,30000.0,€,2012,adrian-partl,...,88.0,01:02:00,f,f,NaN,f,f,100,1,Slovakia
1,1.0,1,2012-06-11,2012-06-17,Slovakia,Clay,30000.0,€,2012,adrian-partl,...,195.0,02:43:00,t,f,NaN,t,f,100,0,Slovakia
2,2.0,2,2012-06-11,2012-06-17,Slovakia,Clay,30000.0,€,2012,agustin-velotti,...,126.0,01:17:00,f,f,NaN,f,f,100,3,Slovakia
3,3.0,3,2012-06-11,2012-06-17,Slovakia,Clay,30000.0,€,2012,agustin-velotti_marcel-felder,...,NaN,NaN,f,f,NaN,t,t,100,4,Slovakia
4,4.0,4,2012-06-11,2012-06-17,Slovakia,Clay,30000.0,€,2012,alessandro-giannessi,...,194.0,02:32:00,t,f,1,t,f,100,3,Slovakia


In [7]:
(matches_data_latest.index - matches_data_latest['Unnamed: 0.1']).sum()

7500150000.0

In [8]:
# create a small sample so we can establish working without using the whole dataset:
# matches_df.to_csv(matches_data_path)

In [9]:
display(matches_df.info()) # 50_000 rows is <21mb in memory

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 55 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Unnamed: 0                            50000 non-null  int64  
 1   start_date                            50000 non-null  object 
 2   end_date                              48967 non-null  object 
 3   location                              50000 non-null  object 
 4   court_surface                         50000 non-null  object 
 5   prize_money                           15537 non-null  float64
 6   currency                              15537 non-null  object 
 7   year                                  50000 non-null  int64  
 8   player_id                             50000 non-null  object 
 9   player_name                           10298 non-null  object 
 10  opponent_id                           50000 non-null  object 
 11  opponent_name  

None

In [10]:
matches_df.columns

Index(['Unnamed: 0', 'start_date', 'end_date', 'location', 'court_surface',
       'prize_money', 'currency', 'year', 'player_id', 'player_name',
       'opponent_id', 'opponent_name', 'tournament', 'round', 'num_sets',
       'sets_won', 'games_won', 'games_against', 'tiebreaks_won',
       'tiebreaks_total', 'serve_rating', 'aces', 'double_faults',
       'first_serve_made', 'first_serve_attempted', 'first_serve_points_made',
       'first_serve_points_attempted', 'second_serve_points_made',
       'second_serve_points_attempted', 'break_points_saved',
       'break_points_against', 'service_games_won', 'return_rating',
       'first_serve_return_points_made', 'first_serve_return_points_attempted',
       'second_serve_return_points_made',
       'second_serve_return_points_attempted', 'break_points_made',
       'break_points_attempted', 'return_games_played', 'service_points_won',
       'service_points_attempted', 'return_points_won',
       'return_points_attempted', 'total_point

In [11]:
# there are a bunch of columns we can avoid using, certainly initially
# looking at the datatypes, there is probably a lot of room for memory reduction

# also some of key data only shows 10298 rows - it might well be that the full data is only 10298 rows

In [12]:
# NOTE from Pandas: columns 5, 8, 10, 45, 48 have mixed datatypes:
# thats:
mixed_datatypes = ['currency', 'player_name', 'opponent_name', 'duration', 'seed']

In [13]:
matches_df.player_name.count()

10298

In [14]:
check_completeness = {'start_date': 0, 'end_date': 0, 'player_name': 0, 'serve_rating': 0, 'duration': 0}
# list(check_completeness.keys())
for i in check_completeness:
    print(i)

start_date
end_date
player_name
serve_rating
duration


In [15]:
for i in check_completeness:
    count = matches_df[i].count()
    check_completeness[i] += count
display(check_completeness)

{'start_date': 50000,
 'end_date': 48967,
 'player_name': 10298,
 'serve_rating': 10298,
 'duration': 10298}

In [16]:
# to go through all the sample data set

start = time.time()

ave_bytes = 0
count = 0
total_bytes = 0

header = True

# This enumerate function selects repeated chunks of 1,000,000 rows of data 
for index, chunk in enumerate(pd.read_csv(matches_data_path, 
                                            chunksize= 10_000,
                                            # usecols = list(check_completeness.keys()), # can be index based (but not -1): [0, 7]
                                            ),
                              start=1):
    this_bytes = chunk.memory_usage().sum()
    ave_bytes += this_bytes
    total_bytes = total_bytes + this_bytes
    # This inner loop iterates through the rate keys only.Then it does 
    # vectorised selections on the dataframe to select count of each rate key.
    for i in check_completeness:
        count = chunk[i].count()
        check_completeness[i] += count
        
    # make changes
    
    #     
    chunk.to_csv(matches_data_path_latest,
                 header=header, 
                 mode='a',
                 # cols=[['','']]
                 )
    header = False # so only first row gets a header

print(f"Total number of chunks: {index}")
print(f"Total memory, MB: {round(total_bytes/100_000,0):,}")
ave_bytes = ave_bytes / index
print(f"Average MB per loop: {round(ave_bytes/100_000,0):,}")
print(check_completeness)

print(round(time.time() - start,2), ' seconds')

Total number of chunks: 5
Total memory, MB: 220.0
Average MB per loop: 44.0
{'start_date': 100000, 'end_date': 97934, 'player_name': 20596, 'serve_rating': 20596, 'duration': 20596}
3.15  seconds


In [17]:
# now to adjust the datatypes as each chunk is loaded:

start = time.time()

ave_bytes = 0
count = 0
total_bytes = 0

header = True

# This enumerate function selects repeated chunks of 1,000,000 rows of data 
for index, chunk in enumerate(pd.read_csv(matches_data_path 
                                            , chunksize= 10_000
                                            , usecols = ['aces'] # list(check_completeness.keys()), # can be index based (but not -1): [0, 7]
                                            , dtype={
                                            'Name': 'string',
                                            'Age': 'int64',
                                            'Height': 'float64',
                                            'Weight': 'float64'
                                        }),
                              start=1):
    this_bytes = chunk.memory_usage().sum()
    ave_bytes += this_bytes
    total_bytes = total_bytes + this_bytes
    # This inner loop iterates through the rate keys only.Then it does 
    # vectorised selections on the dataframe to select count of each rate key.
    for i in check_completeness:
        count = chunk[i].count()
        check_completeness[i] += count
        
    # make changes
    
    #     
    chunk.to_csv(matches_data_path_latest,
                 header=header, 
                 mode='a',
                 # cols=[['','']]
                 )
    header = False # so only first row gets a header

print(f"Total number of chunks: {index}")
print(f"Total memory, MB: {round(total_bytes/100_000,0):,}")
ave_bytes = ave_bytes / index
print(f"Average bytes per loop: {round(ave_bytes/100_000,0):,}")
print(check_completeness)

print(round(time.time() - start,2), ' seconds')

KeyError: 'start_date'

In [ ]:
# when done with data processing, save back to single file with mode="a" 
# header = True
# for chunk in chunks:

#     chunk.to_csv(os.path.join(folder, new_folder, "new_file_" + filename),
#         header=header, cols=[['TIME','STUFF']], mode='a')

#     header = False

In [ ]:
matches_column_names = matches_df.columns
matches_column_names

Index(['start_date', 'end_date', 'location', 'court_surface', 'prize_money',
       'currency', 'year', 'player_id', 'player_name', 'opponent_id',
       'opponent_name', 'tournament', 'round', 'num_sets', 'sets_won',
       'games_won', 'games_against', 'tiebreaks_won', 'tiebreaks_total',
       'serve_rating', 'aces', 'double_faults', 'first_serve_made',
       'first_serve_attempted', 'first_serve_points_made',
       'first_serve_points_attempted', 'second_serve_points_made',
       'second_serve_points_attempted', 'break_points_saved',
       'break_points_against', 'service_games_won', 'return_rating',
       'first_serve_return_points_made', 'first_serve_return_points_attempted',
       'second_serve_return_points_made',
       'second_serve_return_points_attempted', 'break_points_made',
       'break_points_attempted', 'return_games_played', 'service_points_won',
       'service_points_attempted', 'return_points_won',
       'return_points_attempted', 'total_points_won', 'total

In [ ]:
matches_dtypes = matches_df.dtypes
matches_df_objects = matches_dtypes[matches_dtypes==object]
matches_df_objects = pd.DataFrame(matches_df_objects).reset_index()
matches_df_objects.rename(columns={'index': 'column_name', 0: 'dtype'}, inplace=True)
print(matches_df_objects.columns)
matches_df_objects

Index(['column_name', 'dtype'], dtype='object')


,column_name,dtype
0,start_date,object
1,end_date,object
2,location,object
3,court_surface,object
4,currency,object
5,player_id,object
6,player_name,object
7,opponent_id,object
8,opponent_name,object
9,tournament,object


In [ ]:
matches_date_columns = matches_df_objects[:2]
matches_date_columns

,column_name,dtype
0,start_date,object
1,end_date,object


In [ ]:
middle_cols = matches_df[matches_df.columns[12:20]]
middle_cols

,round,num_sets,sets_won,games_won,games_against,tiebreaks_won,tiebreaks_total,serve_rating
0,2nd Round Qualifying,2,0,3,12,0,0,149.0
1,1st Round Qualifying,3,2,16,12,0,0,249.0
2,Round of 32,2,0,7,12,0,0,225.0
3,Round of 16,3,1,18,22,1,1,NaN
4,Round of 32,3,2,17,15,1,1,276.0
...,...,...,...,...,...,...,...,...
49995,Round of 32,2,0,7,12,0,0,NaN
49996,Finals,2,2,12,3,0,0,NaN
49997,Semi-Finals,2,2,12,7,0,0,NaN
49998,Round of 16,2,2,12,6,0,0,NaN


In [ ]:
# ! pip install polars
# import polars as pl # this will not work on my machine

# !pip install polars-lts-cpu
import polars as pl

In [ ]:
# this crashes, too big 890mb
matches_csv_file = '../raw_data/archive/all_matches.csv'
# matches_df_polar = pl.read_csv()
# matches_df_polar = pl.read_csv_batched(matches_csv_file)
reader = pl.read_csv_batched(

    matches_csv_file,

    try_parse_dates=True,

)  

batches = reader.next_batches(5)  

for df in batches:  

    print(df)
    


shape: (61_559, 54)
┌────────────┬────────────┬──────────┬──────────────┬───┬─────────┬─────────┬───────────┬──────────┐
│ start_date ┆ end_date   ┆ location ┆ court_surfac ┆ … ┆ doubles ┆ masters ┆ round_num ┆ nation   │
│ ---        ┆ ---        ┆ ---      ┆ e            ┆   ┆ ---     ┆ ---     ┆ ---       ┆ ---      │
│ date       ┆ date       ┆ str      ┆ ---          ┆   ┆ str     ┆ i64     ┆ i64       ┆ str      │
│            ┆            ┆          ┆ str          ┆   ┆         ┆         ┆           ┆          │
╞════════════╪════════════╪══════════╪══════════════╪═══╪═════════╪═════════╪═══════════╪══════════╡
│ 2012-06-11 ┆ 2012-06-17 ┆ Slovakia ┆ Clay         ┆ … ┆ f       ┆ 100     ┆ 1         ┆ Slovakia │
│ 2012-06-11 ┆ 2012-06-17 ┆ Slovakia ┆ Clay         ┆ … ┆ f       ┆ 100     ┆ 0         ┆ Slovakia │
│ 2012-06-11 ┆ 2012-06-17 ┆ Slovakia ┆ Clay         ┆ … ┆ f       ┆ 100     ┆ 3         ┆ Slovakia │
│ 2012-06-11 ┆ 2012-06-17 ┆ Slovakia ┆ Clay         ┆ … ┆ t       ┆ 100

In [ ]:
df.dtypes

[Date,
 Date,
 String,
 String,
 Int64,
 String,
 Int64,
 String,
 String,
 String,
 String,
 String,
 String,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 Int64,
 String,
 String,
 String,
 String,
 String,
 String,
 Int64,
 Int64,
 String]

In [ ]:
len(df.columns)

54